```shell
uv pip install -qU "semantic-router[local]"
```

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
import itertools
import json
import os
import random
from datetime import datetime
from textwrap import dedent
from typing import List

import matplotlib.pyplot as plt
import numpy as np
from fastembed import TextEmbedding
from gait import (
    FEL,
    Column,
    Layer,
    Layers,
    a_message,
    s_message,
    u_message,
)
from litellm import completion
from pydantic import BaseModel
from rich.pretty import pprint
from semantic_router import Route
from semantic_router.encoders import FastEmbedEncoder
from sentence_transformers import SentenceTransformer
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from tqdm.notebook import tqdm, trange

In [ ]:
layers = Layers.load(os.path.expanduser("~/layers.json"))

In [ ]:
for l,r in itertools.combinations_with_replacement(layers.layers,2):
    print(l.name,r.name)

In [ ]:
fel = FEL(layers.layers)

In [ ]:
attributes = fel.attributes()

In [ ]:
def layer_to_route(l: Layer, r: Layer) -> Route:
    same = l.name == r.name
    utterances = list(
        {
            fel.create_line_2(
                layer_lhs=l,
                layer_rhs=r,
            ).line
            for _ in range(80 if same else 40)
        }
    )
    if not same:
        utterances.extend(list(
        {
            fel.create_line_2(
                layer_lhs=r,
                layer_rhs=l,
            ).line
            for _ in range(40)
        }            
        ))
    return Route(
        name=f"{l.name}:{r.name}",
        utterances=utterances,
    )

In [ ]:
routes = [
    # layer_to_route(l, r) for l, r in itertools.product(layers.layers, layers.layers),
    layer_to_route(l, r) for l, r in itertools.combinations_with_replacement(layers.layers,2)
]

In [ ]:
encoder = FastEmbedEncoder(name="BAAI/bge-base-en-v1.5")

In [ ]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes)

In [ ]:
model = "phi4:14b-q8_0"

In [ ]:
system = dedent(
    f"""
You are an AI specialized in creating variations of geospatial and non-geospatial queries.
Your task is to generate one variation of a user's prompt, incorporating spatial relations, non-spatial relations, and spatial layer filter attributes as appropriate.
Adapt to the context while maintaining logical consistency and relevance to the original query.

Guidelines:
1. Analyze the original query to identify spatial and non-spatial elements.
2. Create ONE variation that rephrases the query with a distinct focus or intent.
3. Incorporate relevant spatial relations (e.g., "intersects," "within," "near"), non-spatial relations (e.g., attribute-based filtering, temporal aspects), and spatial layer filter attributes based on the provided schema.
4. Use clear, concise language in your variation.
5. Maintain the original query's core intent and logic.
6. Do not add spatial context if it doesn't exist in the original prompt.
7. Output only the variation without explanations.
8. DO NOT PLACE OR ADD QUOTES AROUND TEXTURAL VALUES.

If the query lacks clear spatial or non-spatial elements, focus on enhancing the existing elements or clarifying the intent.

Schema for spatial layers:
{attributes}

Respond with only the variation, without additional commentary.
"""
).strip()

In [ ]:
def variation(content: str) -> str:
    response = completion(
        model=f"ollama_chat/{model}",
        temperature=0.35,
        messages=[
            {
                "role": "system",
                "content": system,
            },
            {
                "role": "user",
                "content": "Find pipelines where dimension < 20.0 that are inside wellbores",
            },
            {
                "role": "assistant",
                "content": "Identify pipelines with dimensions less than 20.0 meters that intersect with wellbore locations",
            },
            {
                "role": "user",
                "content": "Identify all discoveries with discovery type > 3 and discovery current activity state is production ceased",
            },
            {
                "role": "assistant",
                "content": "Locate all discoveries where the discovery type exceeds 3 and the discovery's current activity status is marked as production ceased",
            },
            {
                "role": "user",
                "content": "Find pipelines that are inside discoveries where field type is not like gas",
            },
            {
                "role": "assistant",
                "content": "Locate pipelines within discoveries where the field type does not include gas",
            },
            {
                "role": "user",
                "content": content,
            },
        ],
        # stop=[
        #     "```",
        # ],
    )
    return response.choices[0].message.content

In [ ]:
fel_line = fel.create_line_2()
# var_line = variation(fel_line.line)
print(fel_line.line)
# print(var_line)
print(f"{fel_line.fel.layer1}:{fel_line.fel.layer2}", rl(fel_line.line).name)